In [1]:
# from http://brandonrose.org/clustering

In [7]:
import pandas
from sklearn import feature_extraction
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
import nltk

punct = set(string.punctuation)
#my_stopwords = ["a","as","able","about","above","according","accordingly","across","actually","after","afterwards","again","against","aint","all","allow","allows","almost","alone","along","already","also","although","always","am","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","arent","around","as","aside","ask","asking","associated","at","available","away","awfully","be","became","because","become","becomes","becoming","been","before","beforehand","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","cmon","cs","came","can","cant","cannot","cant","cause","causes","certain","certainly","changes","clearly","co","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldnt","course","currently","definitely","described","despite","did","didnt","different","do","does","doesnt","doing","dont","done","down","downwards","during","each","edu","eg","eight","either","else","elsewhere","enough","entirely","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","far","few","fifth","first","five","followed","following","follows","for","former","formerly","forth","four","from","further","furthermore","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","had","hadnt","happens","hardly","has","hasnt","have","havent","having","he","hes","hello","help","hence","her","here","heres","hereafter","hereby","herein","hereupon","hers","herself","hi","him","himself","his","hither","hopefully","how","howbeit","however","id","ill","im","ive","ie","if","ignored","immediate","in","inasmuch","inc","indeed","indicate","indicated","indicates","inner","insofar","instead","into","inward","is","isnt","it","itd","itll","its","its","itself","just","keep","keeps","kept","know","known","knows","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","little","look","looking","looks","ltd","mainly","many","may","maybe","me","mean","meanwhile","merely","might","more","moreover","most","mostly","much","must","my","myself","name","namely","nd","near","nearly","necessary","need","needs","neither","never","nevertheless","new","next","nine","no","nobody","non","none","noone","nor","normally","not","nothing","novel","now","nowhere","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","only","onto","or","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","own","particular","particularly","per","perhaps","placed","please","plus","possible","presumably","probably","provides","que","quite","qv","rather","rd","re","really","reasonably","regarding","regardless","regards","relatively","respectively","right","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","she","should","shouldnt","since","six","so","some","somebody","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","ts","take","taken","tell","tends","th","than","thank","thanks","thanx","that","thats","thats","the","their","theirs","them","themselves","then","thence","there","theres","thereafter","thereby","therefore","therein","theres","thereupon","these","they","theyd","theyll","theyre","theyve","think","third","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","to","together","too","took","toward","towards","tried","tries","truly","try","trying","twice","two","un","under","unfortunately","unless","unlikely","until","unto","up","upon","us","use","used","useful","uses","using","usually","value","various","very","via","viz","vs","want","wants","was","wasnt","way","we","wed","well","were","weve","welcome","well","went","were","werent","what","whats","whatever","when","whence","whenever","where","wheres","whereafter","whereas","whereby","wherein","whereupon","wherever","whether","which","while","whither","who","whos","whoever","whole","whom","whose","why","will","willing","wish","with","within","without","wont","wonder","would","wouldnt","yes","yet","you","youd","youll","youre","youve","your","yours","yourself","yourselves","zero"]
#my_stopwords = ["player","players"]
my_stopwords = []

In [8]:
d = pandas.read_csv("/Users/michaelshea/Documents/dm_survey_data/dm_survey_clean.csv", encoding = 'iso-8859-1')
tips = d["Favorite Trick"]
tips = list(set(tips))
tips = [tip.strip() for tip in list(tips) if tip == tip]
len(tips)

4046

In [9]:
def tokenize(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    text = "".join([ch for ch in text if ch not in punct and not ch.isdigit()]).lower()
    text = [wordnet_lemmatizer.lemmatize(x) for x in text.split() if x not in my_stopwords]
    return(text)

In [15]:
tfidf_vectorizer = TfidfVectorizer(max_df=.8, max_features=200000,
                                 min_df=5, stop_words='english',
                                 use_idf=True, tokenizer=tokenize, ngram_range=(1,5))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(tips) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 1.03 s, sys: 84.1 ms, total: 1.11 s
Wall time: 1.12 s
(4046, 1538)


In [16]:
terms = tfidf_vectorizer.get_feature_names()

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [20]:
from sklearn.cluster import KMeans
cluster_count = 8
km = KMeans(n_clusters=cluster_count)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

pd = pandas.DataFrame({"tips":tips, "clusters":clusters})
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(cluster_count):
    print("Cluster %d" % i, end='')
    print(" (%d tips): " % pd[pd["clusters"] == i]["clusters"].value_counts(), end="")
    for ind in order_centroids[i, :100]: #replace 6 with n words per cluster
        print(' %s' % terms[ind], end=',')
    print()
    print()

    

Cluster 0 (427 tips):  game, session, use, player, like, rule, thing, time, card, initiative, combat, make, cool, world, npc, run, story, dont, dm, idea, start, encounter, know, way, think, help, prep, pc, try, rule cool, best, play, lot, want, note, monster, people, adventure, create, cliffhanger, feel, trick, end, possible, just, campaign, track, work, need, great, inspiration, build, roll, use player, music, good, little, map, book, ha, really, player game, table, ive, come, plan, allow, game player, ask, outline, character, running, end session, set, moment, point, im, stuff, background, let, having, prepared, want game, write, event, important, great game, previous, remember, prepare, scene, using, small, flow, going, run game, start session, make game, different, player session,

Cluster 1 (742 tips):  player, want, story, player want, listen, world, idea, letting, listen player, letting player, action, know, play, try, choice, work, know player, roll, thing, listening, game, npc

In [35]:
pd = pd.sort_values(by=["clusters"])
pd.to_csv("/Users/michaelshea/Documents/dm_survey_data/dm_survey_tips_clusters.csv", index=False)